In [27]:
from gensim_preprocessing import corpus, dictionary, texts
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [28]:
bigrams_phrases = gensim.models.Phrases(texts, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[texts], threshold=100)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(textss):
    return [bigram[doc] for doc in textss]

def make_trigrams(textss):
    return [trigram[bigram[doc]] for doc in textss]

data_bigrams = make_bigrams(texts)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[0])


['attribuer', 'noyenn', 'chave', 'sportif', 'chave', 'unette', 'débarrasser', 'examen', 'sportif', 'annexe', 'intercommunal', 'chabre', 'remplacement', 'février', 'intercommunal', 'glace', 'mars', 'féli', 'sportif', 'cigm', 'losan', 'club', 'atténance', 'étable', 'loco', 'cercle', 'arme', 'centaine', 'junior', 'tir', 'section', 'tireur', 'comprimé', 'centaine', 'tireur', 'hall', 'adapter', 'proximité', 'cigm', 'loger', 'congé', 'syndical', 'désirer', 'solde', 'comptable', 'partel', 'patrimoine', 'patrimoine', 'remplacement', 'abandonner', 'commentaire', 'féli', 'ressortir', 'desservir', 'usager', 'motorisé', 'cigm', 'quinzaine', 'parc', 'aménager', 'club', 'stockage', 'partel', 'inconstructible', 'aménager', 'projeter', 'stockage', 'cigm', 'conception', 'polyvalent', 'hall', 'occasionnel', 'récent', 'championnat', 'curling', 'gymnastique', 'pratiquer', 'tir_arc', 'longueur', 'club', 'séparer', 'paroi', 'mobile', 'insonoriser', 'stockage', 'dalle', 'n', 'club', 'embêter', 'toit', 'suppo

In [ ]:
# TF-IDF REMOVAL

In [29]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts_removal = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts_removal]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word )

low_value = 0.03
words = []
words_missing_in_tfidf = []


[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 5), (17, 3), (18, 1), (19, 1)]


In [15]:
print(texts[0][0:20])

['attribuer', 'noyenn', 'chave', 'sportif', 'chave', 'unette', 'débarrasser', 'examen', 'sportif', 'annexe', 'intercommunal', 'chabre', 'remplacement', 'février', 'intercommunal', 'glace', 'mars', 'féli', 'sportif', 'cigm']


## Vizualizing the data

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word=dictionary,
                                           num_topics=15,
                                           random_state = 100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [12]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds="mmds", R=30)
vis

/Users/mariemccormick/AciduleApp/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.130660  0.231263       1        1  13.793573
0     -0.047371  0.186373       2        1  13.040244
2     -0.294785  0.098816       3        1  10.852219
8      0.034266  0.277048       4        1   8.889331
9      0.308758  0.005224       5        1   7.598577
4     -0.172795  0.041113       6        1   7.242095
6      0.093059 -0.329252       7        1   7.003259
14     0.213069  0.208529       8        1   6.489485
3     -0.168408 -0.251298       9        1   5.659223
1     -0.300724 -0.096780      10        1   5.498888
7      0.141652  0.110745      11        1   5.159473
5      0.242284 -0.135564      12        1   4.878939
11    -0.042463 -0.192278      13        1   2.902974
10     0.119615 -0.120101      14        1   0.989555
13     0.004502 -0.033838      15        1   0.002164, topic_info=              Term        Freq       Total Category  logprob  loglift
236         déchet  553.000000  553.000000  Default  30.0000  30.0000
1135       immigré  314.000000  314.000000  Default  29.0000  29.0000
996    consultatif  258.000000  258.000000  Default  28.0000  28.0000
462          loyer  268.000000  268.000000  Default  27.0000  27.0000
727           site  276.000000  276.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
392   incinération    0.000343  106.437320  Topic15  -8.7418  -1.9056
1856       défense    0.000349  168.119111  Topic15  -8.7230  -2.3439
2362       portier    0.000343  141.495881  Topic15  -8.7401  -2.1886
3101     locataire    0.000344  164.202332  Topic15  -8.7393  -2.3366
1923      espagnol    0.000332  145.585994  Topic15  -8.7739  -2.2509

[1044 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
12009     12  0.861903     abeille
7679      14  0.536216    abouitir
5          1  0.303453  abstention
5          2  0.182072  abstention
5          3  0.157796  abstention
...      ...       ...         ...
4788       5  0.079155         île
4788       7  0.079155         île
4788       9  0.197888         île
4788      12  0.316621         île
4788      14  0.197888         île

[2593 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 1, 3, 9, 10, 5, 7, 15, 4, 2, 8, 6, 12, 11, 14])